In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time
import os
from pathlib import Path

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)




def weight_variable(shape,astype):
    initial = tf.truncated_normal(shape, stddev=0.01, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def bias_variable(shape,astype):
    initial = tf.constant(0.0, shape=shape, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def nn_example(e, b, data_type, filepath):
    learning_rate = 0.5
    epochs = e
    batch_size = b
    input_feature_count = 784
    out_classes = 10
    data_type = data_type
    
    # Neural network hidden layer variables
    h1 = 50
    h2 = 20

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(data_type, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(data_type, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(data_type)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], data_type)
    b_fc1 = bias_variable([h1], data_type)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(data_type)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],data_type)
    b_fc2 = bias_variable([h2], data_type)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],data_type)
    b_fc3 = bias_variable([out_classes], data_type)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, data_type))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)
    
    saver = tf.train.Saver()

    merged = tf.summary.merge([accuracy_sum])
    writer = tf.summary.FileWriter(r'C:\Users\anant\dev\repos\HPA')
    
    test_accuracy = 0
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = int(len(mnist.train.labels) / batch_size)
        start_time = time.time()
        for epoch in range(max(epochs)+1):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                train_acc,_, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
                arr_accuracy = [];
                time_elapsed = [];
                nw_size = [];
            if(epoch in epochs):
                elapsed = time.time() - start_time
                filepath = "D:\\UCI\\HPA_Project\\weights_data\\epoch_"+ str(epoch)+ '_batch_' +str(batch_size)
                save_path = saver.save(sess, filepath)
                mfilepath = filepath + ".data-00000-of-00001"
                file = Path(mfilepath)
                size = file.stat().st_size
                test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
                arr_accuracy.append(test_accuracy)
                time_elapsed.append(elapsed)
                nw_size.append(size)
                print(epoch, b, test_accuracy, elapsed)
    return arr_accuracy, time_elapsed, nw_size

if __name__ == "__main__":
    data_type = tf.float32
    batch_size = [50, 100, 200, 500, 1000]
    epochs = [5,10, 15, 20] #, 30, 50] #, 80, 100, 150, 200, 300, 500, 700, 1000]
    num_epochs = len(epochs)
    num_bsize = len(batch_size)
    accuracy_mat = [[] for i in range(num_bsize)]
    elapsed_time = [[] for i in range(num_bsize)]
    size_mat = [[] for i in range(num_bsize)]
    for j, b in enumerate(batch_size):
        accuracy_mat[j], elapsed_time[j], size_mat[j] = nn_example(epochs, b, data_type, filepath)
        print()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
epochs = [20, 30, 50, 80, 100, 150, 200, 300, 500, 700, 1000]
for epoch in range(max(epochs)+1):
    if(epoch in epochs):
        print(epoch)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time
import os
from pathlib import Path

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
def weight_variable(shape,astype):
    initial = tf.truncated_normal(shape, stddev=0.01, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def bias_variable(shape,astype):
    initial = tf.constant(0.0, shape=shape, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def nn_example(e, batch, data_type, filepath):
    # Python optimisation variables
    learning_rate = 0.5
    epochs = e
    batch_size = b
    input_feature_count = 784
    out_classes = 10
    data_type = data_type
    
    # Neural network hidden layer variables
    h1 = 50
    h2 = 20

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(data_type, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(data_type, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(data_type)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], data_type)
    b_fc1 = bias_variable([h1], data_type)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(data_type)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],data_type)
    b_fc2 = bias_variable([h2], data_type)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],data_type)
    b_fc3 = bias_variable([out_classes], data_type)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, data_type))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)
    
    saver = tf.train.Saver()

    merged = tf.summary.merge([accuracy_sum])
    writer = tf.summary.FileWriter(r'C:\Users\anant\dev\repos\HPA')
    
    test_accuracy = 0
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = int(len(mnist.train.labels) / batch_size)
        start_time = time.time()
        for epoch in range(epochs):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                train_acc,_, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
#             print("Epoch:", (epoch + 1), "Train accuracy =", "{:.3f}".format(train_acc), "cost =", "{:.3f}".format(avg_cost))
#             summary = sess.run(merged, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
#             writer.add_summary(summary, epoch)
        elapsed = time.time() - start_time
#         print("\nTraining complete!")
#         writer.add_graph(sess.graph)
        save_path = saver.save(sess, filepath)
        mfilepath = filepath + ".data-00000-of-00001"
        file = Path(mfilepath)
        size = file.stat().st_size
        test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
#         print('For e={} & b={}, test_accuracy={}'.format(epoch, batch_size, test_accuracy))
    return test_accuracy, elapsed, size

if __name__ == "__main__":
    data_type = tf.float16
    epochs = [5, 10, 15, 20, 50]
#     epochs = [50]
    batch_size = [50, 100, 200, 500, 1000]
    
    num_epochs = len(epochs)
    num_bsize = len(batch_size)
    accuracy_mat = [[0 for j in range(num_bsize)] for i in range(num_epochs)]
    elapsed_time = [[0 for j in range(num_bsize)] for i in range(num_epochs)]
    size_mat = [[0 for j in range(num_bsize)] for i in range(num_epochs)]
    for i, e in enumerate(epochs):
        for j, b in enumerate(batch_size):
            filepath = "D:\\UCI\\HPA_Project\\weights_data\\epoch_"+ str(e)+ '_batch_' +str(b)
            accuracy_mat[i][j], elapsed_time[i][j], size_mat[i][j]= nn_example(e, b, data_type, filepath)
            print(epochs[i],batch_size[j],accuracy_mat[i][j], elapsed_time[i][j])
        print()


In [ ]:
5 50 0.098022 28.80020833015442
5 100 0.098022 16.516690492630005
5 200 0.098022 10.767619371414185
5 500 0.098022 6.33238673210144
5 1000 0.098022 4.642341375350952

10 50 0.098022 63.0987548828125
10 100 0.098022 32.40916156768799
10 200 0.098022 21.14822745323181
10 500 0.098022 11.47575831413269
10 1000 0.098022 10.29037618637085

15 50 0.098022 87.81630396842957
15 100 0.098022 48.233941316604614
15 200 0.098022 28.5248761177063
15 500 0.098022 16.72200894355774
15 1000 0.098022 13.220077276229858

In [ ]:
def weight_variable(shape,astype):
    initial = tf.truncated_normal(shape, stddev=0.01, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def bias_variable(shape,astype):
    initial = tf.constant(0.0, shape=shape, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def nn_example(e, batch):
    # Python optimisation variables
    learning_rate = 0.5
    epochs = e
    batch_size = b
    input_feature_count = 784
    out_classes = 10
    
    # Neural network hidden layer variables
    h1 = 50
    h2 = 20

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(tf.float64, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(tf.float64, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(tf.float64)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], tf.float64)
    b_fc1 = bias_variable([h1], tf.float64)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float64)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],tf.float64)
    b_fc2 = bias_variable([h2], tf.float64)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],tf.float64)
    b_fc3 = bias_variable([out_classes], tf.float64)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)

    merged = tf.summary.merge([accuracy_sum])
    writer = tf.summary.FileWriter(r'C:\Users\anant\dev\repos\HPA')
    
    test_accuracy = 0
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = int(len(mnist.train.labels) / batch_size)
        start_time = time.time()
        for epoch in range(epochs):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                train_acc,_, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
#             print("Epoch:", (epoch + 1), "Train accuracy =", "{:.3f}".format(train_acc), "cost =", "{:.3f}".format(avg_cost))
#             summary = sess.run(merged, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
#             writer.add_summary(summary, epoch)
        elapsed = time.time() - start_time
#         print("\nTraining complete!")
#         writer.add_graph(sess.graph)
        test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
#         print('For e={} & b={}, test_accuracy={}'.format(epoch, batch_size, test_accuracy))
    return test_accuracy, elapsed

if __name__ == "__main__":
    epochs = [1, 2, 3, 5, 10]
    batch_size = [10, 20, 50, 100, 200, 500, 1000]
    
    num_epochs = len(epochs)
    num_bsize = len(batch_size)
    accuracy_mat = [[0 for j in range(num_bsize)] for i in range(num_epochs)]
    elapsed_time = [[0 for j in range(num_bsize)] for i in range(num_epochs)]
    for i, e in enumerate(epochs):
        for j, b in enumerate(batch_size):
            accuracy_mat[i][j], elapsed_time[i][j]= nn_example(e, b)
            print(accuracy_mat[i][j], elapsed_time[i][j])
        print()

In [ ]:
def weight_variable(shape,astype):
    initial = tf.truncated_normal(shape, stddev=0.01, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def bias_variable(shape,astype):
    initial = tf.constant(0.0, shape=shape, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def nn_example(e, batch):
    # Python optimisation variables
    learning_rate = 0.5
    epochs = e
    batch_size = b
    input_feature_count = 784
    out_classes = 10
    
    # Neural network hidden layer variables
    h1 = 50
    h2 = 20

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(tf.float16, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(tf.float16, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(tf.float16)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], tf.float16)
    b_fc1 = bias_variable([h1], tf.float16)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float16)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],tf.float16)
    b_fc2 = bias_variable([h2], tf.float16)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],tf.float16)
    b_fc3 = bias_variable([out_classes], tf.float16)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float16))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)

    merged = tf.summary.merge([accuracy_sum])
    writer = tf.summary.FileWriter(r'C:\Users\anant\dev\repos\HPA')
    
    test_accuracy = 0
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = int(len(mnist.train.labels) / batch_size)
        start_time = time.time()
        for epoch in range(epochs):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                train_acc,_, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
#             print("Epoch:", (epoch + 1), "Train accuracy =", "{:.3f}".format(train_acc), "cost =", "{:.3f}".format(avg_cost))
#             summary = sess.run(merged, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
#             writer.add_summary(summary, epoch)
        elapsed = time.time() - start_time
#         print("\nTraining complete!")
#         writer.add_graph(sess.graph)
        test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
#         print('For e={} & b={}, test_accuracy={}'.format(epoch, batch_size, test_accuracy))
    return test_accuracy, elapsed

if __name__ == "__main__":
    epochs = [1, 2, 3, 5, 10]
    batch_size = [10, 20, 50, 100, 200, 500, 1000]
    
    num_epochs = len(epochs)
    num_bsize = len(batch_size)
    accuracy_mat = [[0 for j in range(num_bsize)] for i in range(num_epochs)]
    elapsed_time = [[0 for j in range(num_bsize)] for i in range(num_epochs)]
    for i, e in enumerate(epochs):
        for j, b in enumerate(batch_size):
            accuracy_mat[i][j], elapsed_time[i][j]= nn_example(e, b)
            print(accuracy_mat[i][j], elapsed_time[i][j])
        print()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline
sys.path.append(r'D:\UCI\CS273A-ML\HW4-code') #append path for mltools
sys.path.append(r'D:\UCI\CS273A-ML\Project') #append path for data
np.random.seed(0)

import mltools as ml

In [ ]:
def load_data():
    # Data Loading
    X = np.genfromtxt(r'D:\UCI\CS273A-ML\Project\X_train.txt' , delimiter=None)
    Y = np.genfromtxt(r'D:\UCI\CS273A-ML\Project\Y_train.txt' , delimiter=None)
    X, Y = ml.shuffleData(X,Y)
    
    return X, Y

In [ ]:
X, Y = load_data()

In [ ]:
def read_ml_hw(X, Y):
    onehot_encoder = OneHotEncoder(sparse=False)
    Xtr, Xva, Ytr, Yva = ml.splitData(X, Y)
    Ytr = onehot_encoder.fit_transform(Ytr.reshape(len(Ytr), 1))
    Yva = onehot_encoder.fit_transform(Yva.reshape(len(Yva), 1))
    Xt, Yt = Xtr[:], Ytr[:] # subsample for efficiency (you can go higher)
    Xv, Yv = Xva[:], Yva[:] # subsample for efficiency (you can go higher)
    Xts, params = ml.rescale(Xt) # Normalize the features
#     Xvs, _ = ml.rescale(Xv, params) # Normalize the features
    print(Xts.shape, Yt.shape, Xv.shape, Yv.shape)
    return Xt, Yt, Xv, Yv
    
read_ml_hw(X, Y)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

def weight_variable(shape,astype):
    initial = tf.truncated_normal(shape, stddev=0.01, dtype = astype)
    return tf.Variable(initial, dtype = astype)


def bias_variable(shape,astype):
    initial = tf.constant(0.0, shape=shape, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def read_input():
    inp_file = open(r'C:\Users\anant\dev\repos\HPA\inputs', 'rb')
    inp = pickle.load(inp_file)
    inp_file.close()
    print(inp[:3])
    inp[:,0:2] = inp[:,0:2]*10
    print(inp[:3])
    
    out_file = open(r'C:\Users\anant\dev\repos\HPA\outputs', 'rb')
    out = pickle.load(out_file)
    out_file.close()

    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = onehot_encoder.fit_transform(out.reshape(len(out), 1))

    inp_train, inp_val, onehot_encoded_train, onehot_encoded_val = train_test_split(inp, onehot_encoded)
    print(inp_train.shape, onehot_encoded_train.shape, inp_val.shape, onehot_encoded_val.shape)
    return inp_train, onehot_encoded_train, inp_val, onehot_encoded_val

def read_ml_hw(X, Y):
    onehot_encoder = OneHotEncoder(sparse=False)
    Xtr, Xva, Ytr, Yva = ml.splitData(X, Y)
    Ytr = onehot_encoder.fit_transform(Ytr.reshape(len(Ytr), 1))
    Yva = onehot_encoder.fit_transform(Yva.reshape(len(Yva), 1))
    Xt, Yt = Xtr[:], Ytr[:] # subsample for efficiency (you can go higher)
    Xv, Yv = Xva[:], Yva[:] # subsample for efficiency (you can go higher)
    Xts, params = ml.rescale(Xt) # Normalize the features
#     Xvs, _ = ml.rescale(Xv, params) # Normalize the features
    print(Xts.shape, Yt.shape, Xv.shape, Yv.shape)
    return Xt, Yt, Xv, Yv

def nn_example():
#     inp, out, inp_va, out_va = read_input()
    X, Y = load_data()
    inp, out, inp_va, out_va = read_ml_hw(X, Y)
    no_of_data = inp.shape[0]
    input_feature_count = inp.shape[1]
    out_classes = out.shape[1]
    print('Features={}, classes={}'.format(input_feature_count, out_classes))
    print('Inp_train={}, inp_val={}, out_train={}, out_val={}'.format(inp.shape, inp_va.shape, out.shape, out_va.shape))
    
    # Python optimisation variables
    learning_rate = 0.001
    epochs = 100
    batch_size = 1
    
    # Neural network hidden layer variables
    h1 = 5
    h2 = 3

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(tf.float32, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(tf.float32, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(tf.float32)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], tf.float32)
    b_fc1 = bias_variable([h1], tf.float32)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],tf.float32)
    b_fc2 = bias_variable([h2], tf.float32)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],tf.float32)
    b_fc3 = bias_variable([out_classes], tf.float32)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)

    merged = tf.summary.merge([accuracy_sum])
    writer = tf.summary.FileWriter(r'C:\Users\anant\dev\repos\HPA')
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = no_of_data//batch_size
        print('Total number of batches to train={}'.format(total_batch))
        for epoch in range(epochs):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = inp[i*batch_size:(i+1)*batch_size], out[i*batch_size:(i+1)*batch_size]
                train_acc, _, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
            print("Epoch:", (epoch + 1), "Train accuracy =", "{:.3f}".format(train_acc), "cost =", "{:.3f}".format(avg_cost))
#             summary = sess.run(merged, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
#             writer.add_summary(summary, epoch)

        print("\nTraining complete!")
#         writer.add_graph(sess.graph)
        print(sess.run(accuracy, feed_dict={x: inp_va, y: out_va, keep_prob_input: 1.0, keep_prob: 1.0}))

if __name__ == "__main__":
    nn_example()

Epoch: 1 Train accuracy = 0.650 cost = 1.249
Epoch: 2 Train accuracy = 0.660 cost = 1.231
Epoch: 3 Train accuracy = 0.680 cost = 1.227
Epoch: 4 Train accuracy = 0.690 cost = 1.225
Epoch: 5 Train accuracy = 0.670 cost = 1.223
Epoch: 6 Train accuracy = 0.670 cost = 1.221
Epoch: 7 Train accuracy = 0.670 cost = 1.220
Epoch: 8 Train accuracy = 0.670 cost = 1.219
Epoch: 9 Train accuracy = 0.690 cost = 1.218
Epoch: 10 Train accuracy = 0.660 cost = 1.217

Training complete!
0.692975

## Below cell commented out

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

%matplotlib inline
sys.path.append(r'D:\UCI\CS273A-ML\HW4-code') #append path for mltools
sys.path.append(r'D:\UCI\CS273A-ML\Project') #append path for data
np.random.seed(0)

import mltools as ml

# Data Loading
X = np.genfromtxt(r'D:\UCI\CS273A-ML\Project\X_train.txt' , delimiter=None)
Y = np.genfromtxt(r'D:\UCI\CS273A-ML\Project\Y_train.txt' , delimiter=None)
X, Y = ml.shuffleData(X,Y)

Xtr, Xva, Ytr, Yva = ml.splitData(X, Y)
Xt, Yt = Xtr[:5000], Ytr[:5000] # subsample for efficiency (you can go higher)
Xv, Yv = Xva[:5000], Yva[:5000] # subsample for efficiency (you can go higher)
Xts, params = ml.rescale(Xt) # Normalize the features
Xvs, _ = ml.rescale(Xv, params) # Normalize the features

In [ ]:
def read_input():
    inp_file = open(r'C:\Users\anant\dev\repos\HPA\inputs', 'rb')
    inp = pickle.load(inp_file)
    inp_file.close()
    print(inp[:3])
    inp[:,0:2] = inp[:,0:2]*10
    print(inp[:3])
    
    out_file = open(r'C:\Users\anant\dev\repos\HPA\outputs', 'rb')
    out = pickle.load(out_file)
    out_file.close()

    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = onehot_encoder.fit_transform(out.reshape(len(out), 1))

    inp_train, inp_val, onehot_encoded_train, onehot_encoded_val = train_test_split(inp, onehot_encoded)
    print(inp_train.shape, onehot_encoded_train.shape, inp_val.shape, onehot_encoded_val.shape)
    return inp_train, onehot_encoded_train, inp_val, onehot_encoded_val

In [ ]:
def nn_example():
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

    # Python optimisation variables
    learning_rate = 0.5
    epochs = 10
    batch_size = 100
    input_feature_count = 784
    out_classes = 10
    
    # Neural network hidden layer variables
    h1 = 500
    h2 = 200

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(tf.float32, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(tf.float32, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(tf.float32)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], tf.float32)
    b_fc1 = bias_variable([h1], tf.float32)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],tf.float32)
    b_fc2 = bias_variable([h2], tf.float32)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],tf.float32)
    b_fc3 = bias_variable([out_classes], tf.float32)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)

    merged = tf.summary.merge([accuracy_sum])
    writer = tf.summary.FileWriter(r'C:\Users\anant\dev\repos\HPA')
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = int(len(mnist.train.labels) / batch_size)
        for epoch in range(epochs):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                train_acc,_, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
            print("Epoch:", (epoch + 1), "Train accuracy =", "{:.3f}".format(train_acc), "cost =", "{:.3f}".format(avg_cost))
            summary = sess.run(merged, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
            writer.add_summary(summary, epoch)

        print("\nTraining complete!")
        writer.add_graph(sess.graph)
        print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0}))

if __name__ == "__main__":
    nn_example()

In [ ]:
t

In [ ]:
ans = ifnp.sin(t)

In [ ]:
ans